In [1]:
import polars as pl
import duckdb


In [2]:
# use to get a dict of cols and dtypes to define table fields
# cols = pl.read_csv('data/portal_upload/species-tbl.csv', separator=';').columns
# dtypes = pl.read_csv('data/portal_upload/species-tbl.csv', separator=';').dtypes

In [3]:
# dict(zip(cols, dtypes))

In [4]:
# pl_df = pl.read_csv('data/portal_upload/grants-tbl.csv', separator=";")

In [5]:
# pl_df.glimpse()

In [6]:
con = duckdb.connect('data/lnrs_db.duckdb')

In [7]:
def get_tables_list(con):
    tables = con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()
    # This will print the list of table names
    table_list= [table[0] for table in tables]
    return table_list



In [8]:
table_list = get_tables_list(con)
reversed_table_list = table_list[::-1] # needs reversing to delete lookup tables first to avoid constraint error
table_list

['grants_tbl',
 'areas_tbl',
 'priority_measures_tbl',
 'area_measures_tbl',
 'priorities_tbl',
 'species_tbl',
 'area_funding_schemes_tbl',
 'priorities_measures_lookup_tbl',
 'priorities_areas_measures_lookup_tbl',
 'priorities_areas_lookup_tbl',
 'species_priority_lookup_tbl',
 'species_area_lookup_tbl',
 'priority_measures_grants_lookup_tbl',
 'areas_measures_grants_lookup_tbl']

In [9]:
for table in reversed_table_list:
    con.execute(f"DROP TABLE IF EXISTS {table}")

Grants


In [10]:
con.sql(
    """
CREATE TABLE grants_tbl 
(
grant_id VARCHAR (255) UNIQUE NOT NULL PRIMARY KEY,
id INT,
url VARCHAR (255),
grant_name VARCHAR (255),
grant_focus VARCHAR (255),
grant_scheme VARCHAR (255),
annual_payment VARCHAR (255)
);
"""
)

Areas

In [11]:
con.sql(
    """
CREATE TABLE areas_tbl (
    area_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    area_name VARCHAR,
    area_description VARCHAR,
    area_link VARCHAR
);
    """
)

Funding Schemes

In [12]:
con.sql(
    """
CREATE TABLE area_funding_schemes_tbl 
(id INTEGER UNIQUE NOT NULL PRIMARY KEY,
area_id INTEGER,
funding_schemes VARCHAR,
FOREIGN KEY (area_id) REFERENCES areas_tbl (area_id));
    """
)

Priority Measures

In [13]:
con.sql(
    """
CREATE TABLE priority_measures_tbl(
    priority_measure_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    measure VARCHAR,
    level_of_ambition VARCHAR,
    land_type VARCHAR,
    stakeholder VARCHAR,
    link_to_further_guidance VARCHAR
);
    """
)

Area Measures

In [14]:
con.sql(
    """
CREATE TABLE area_measures_tbl(
    area_measure_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    measure VARCHAR,
    level_of_ambition VARCHAR,
    land_type VARCHAR,
    stakeholder VARCHAR
);
    """
)

Priorities

In [15]:
con.sql(
    """
CREATE TABLE priorities_tbl
(priority_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
theme VARCHAR,
biodiversity_priority VARCHAR,
simplified_biodiversity_priority VARCHAR
);
    """
)

Species

In [16]:
con.sql(
    '''
CREATE TABLE species_tbl(
 species_id INTEGER UNIQUE NOT NULL PRIMARY KEY,
 taxa VARCHAR,
 common_name VARCHAR,
 assemblage VARCHAR,
 usage_key INTEGER,
 scientific_name VARCHAR,
 canonical_name VARCHAR,
 status VARCHAR,
 kingdom VARCHAR,
 phylum VARCHAR,
 "order" VARCHAR,
 family VARCHAR,
 genus VARCHAR,
 species VARCHAR,
 kingdom_key INTEGER,
 phylum_key INTEGER,
 class_key INTEGER,
 order_key INTEGER,
 family_key INTEGER,
 genus_key INTEGER,
 species_key INTEGER,
 synonym BOOL,
 class VARCHAR,
 accepted_usage_key INTEGER,
 verbatim_name VARCHAR,
 gbif_species_url VARCHAR
    
);
    '''
)

Priorities Measures Lookup

In [17]:
con.sql(
    """
CREATE TABLE priorities_measures_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    priority_id INTEGER,
    priority_measure_id INTEGER,
    FOREIGN KEY (priority_id) REFERENCES priorities_tbl (priority_id),
    FOREIGN KEY (priority_measure_id) REFERENCES priority_measures_tbl (priority_measure_id)
);
    """
)

Priorities Areas Measures Lookup

In [18]:
con.sql(
    """
CREATE TABLE priorities_areas_measures_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    priority_id INTEGER,
    area_id INTEGER,
    area_measure_id INTEGER,
    FOREIGN KEY (area_measure_id) REFERENCES area_measures_tbl (area_measure_id),
    FOREIGN KEY (priority_id) REFERENCES priorities_tbl (priority_id),
    FOREIGN KEY (area_id) REFERENCES areas_tbl (area_id)
);
    """
)

Priorities Areas Lookup

In [19]:
con.sql(
    """
CREATE TABLE priorities_areas_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    priority_id INTEGER,
    area_id INTEGER,
    FOREIGN KEY (area_id) REFERENCES areas_tbl (area_id),
    FOREIGN KEY (priority_id) REFERENCES priorities_tbl (priority_id)
);
    """
)

Species Priority Lookup

In [20]:
con.sql(
    """
CREATE TABLE species_priority_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    priority_id INTEGER,
    species_id INTEGER,
    FOREIGN KEY (species_id) REFERENCES species_tbl (species_id),
    FOREIGN KEY (priority_id) REFERENCES priorities_tbl (priority_id)
);
    """
)

Species Area Lookup

In [21]:
con.sql(
    """
CREATE TABLE species_area_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    species_id INTEGER,
    area_id INTEGER,
    FOREIGN KEY (species_id) REFERENCES species_tbl (species_id),
    FOREIGN KEY (area_id) REFERENCES areas_tbl (area_id)
);
    """
)

Priority Measures Grants Lookup

In [22]:
con.sql(
    """
CREATE TABLE priority_measures_grants_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    priority_measure_id INTEGER,
    grant_id VARCHAR (255),
    FOREIGN KEY (priority_measure_id) REFERENCES priority_measures_tbl (priority_measure_id),
    FOREIGN KEY (grant_id) REFERENCES grants_tbl (grant_id)
);
    """
)

Areas Measures Grants Lookup

In [23]:
con.sql(
    """
CREATE TABLE areas_measures_grants_lookup_tbl(
    id INTEGER UNIQUE NOT NULL PRIMARY KEY,
    area_measure_id INTEGER,
    grant_id VARCHAR (255),
    FOREIGN KEY (area_measure_id) REFERENCES area_measures_tbl (area_measure_id),
    FOREIGN KEY (grant_id) REFERENCES grants_tbl (grant_id)
);
    """
)

In [24]:
get_tables_list(con)

['grants_tbl',
 'areas_tbl',
 'area_funding_schemes_tbl',
 'priority_measures_tbl',
 'area_measures_tbl',
 'priorities_tbl',
 'species_tbl',
 'priorities_measures_lookup_tbl',
 'priorities_areas_measures_lookup_tbl',
 'priorities_areas_lookup_tbl',
 'species_priority_lookup_tbl',
 'species_area_lookup_tbl',
 'priority_measures_grants_lookup_tbl',
 'areas_measures_grants_lookup_tbl']

In [25]:
try:
    con.execute("BEGIN TRANSACTION;")
    con.execute("COPY grants_tbl FROM 'data/portal_upload/grants-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY areas_tbl FROM 'data/portal_upload/areas-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY area_funding_schemes_tbl FROM 'data/portal_upload/area-funding-schemes-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priority_measures_tbl FROM 'data/portal_upload/priority-measures-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY area_measures_tbl FROM 'data/portal_upload/area-measures-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priorities_tbl FROM 'data/portal_upload/priorities-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY species_tbl FROM 'data/portal_upload/species-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priorities_measures_lookup_tbl FROM 'data/portal_upload/priorities-measures-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priorities_areas_measures_lookup_tbl FROM 'data/portal_upload/priorities-areas-measures-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priorities_areas_lookup_tbl FROM 'data/portal_upload/priorities-areas-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY species_priority_lookup_tbl FROM 'data/portal_upload/species-priority-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY species_area_lookup_tbl FROM 'data/portal_upload/species-area-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY priority_measures_grants_lookup_tbl FROM 'data/portal_upload/priority-measures-grants-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COPY areas_measures_grants_lookup_tbl FROM 'data/portal_upload/areas-measures-grants-lookup-tbl.csv'(DELIMITER ';', HEADER);")
    con.execute("COMMIT;")
except Exception as e:
    # If an error occurs, rollback the transaction
    con.execute("ROLLBACK;")
    print(f"Transaction rolled back due to an error: {e}")

In [26]:
con.sql('SELECT * FROM species_tbl')

┌────────────┬───────────────────┬──────────────────────┬───┬──────────────────────┬──────────────────────┐
│ species_id │       taxa        │     common_name      │ … │    verbatim_name     │   gbif_species_url   │
│   int32    │      varchar      │       varchar        │   │       varchar        │       varchar        │
├────────────┼───────────────────┼──────────────────────┼───┼──────────────────────┼──────────────────────┤
│          1 │ Bee, wasps, flies │ Long-horned bee      │ … │ Eucera longicornis   │ https://www.gbif.o…  │
│          2 │ Bee, wasps, flies │ Western Wood-vase …  │ … │ Myolepta potens      │ https://www.gbif.o…  │
│          3 │ Beetle            │ Glow worm            │ … │ Lampyris noctiluca   │ https://www.gbif.o…  │
│          4 │ Bird              │ Swift                │ … │ Apus apus            │ https://www.gbif.o…  │
│          5 │ Bird              │ Swallow              │ … │ Hirundo rustica      │ https://www.gbif.o…  │
│          6 │ Bird         

In [27]:
species_df = con.sql(
    """
SELECT biodiversity_priority, common_name 
FROM priorities_tbl
JOIN species_priority_lookup_tbl ON priorities_tbl.priority_id = species_priority_lookup_tbl.priority_id
JOIN species_tbl ON species_priority_lookup_tbl.species_id = species_tbl.species_id
    """
).pl()

In [28]:
species_df

biodiversity_priority,common_name
str,str
"""There is a gre…","""Long-horned be…"
"""There are more…","""Long-horned be…"
"""Existing speci…","""Long-horned be…"
"""The farmed lan…","""Long-horned be…"
"""Our designated…","""Western Wood-v…"
"""There is more …","""Western Wood-v…"
"""There are more…","""Glow worm"""
"""Existing speci…","""Glow worm"""
"""Our towns and …","""Swift"""


In [29]:
con.close()